# Functional Programming Techniques in Python: Part 3

In [Part 1](https://mpkocher.github.io/2019/01/30/Functional-Python-Part-1/), we introduced core Functional Programming techniques in Python.

In [Part 2](https://mpkocher.github.io/2019/02/02/Functional-Python-Part-2/), we're designed a REST API client using Functional Programming Techniques (FPT). 

In Part 3, we're going to do another concrete example and build a commandline tool interface. 

This will focus on ironing out some friction points when interfacing with the standard lib (e.g., argparse) while also aiming for a DRY model. 


In [1]:
import datetime
import functools
import itertools
import argparse
import sys
import logging

In [2]:
print("Today is {}".format(datetime.datetime.now()))

Today is 2019-02-21 16:59:11.100236


# Utils

In [Part 1](https://mpkocher.github.io/2019/01/30/Functional-Python-Part-1/), we defined a few utils. We'll reproduce the `compose` util here. 

In [3]:
def compose(*funcs):
    """Functional composition
    [f, g, h] will be f(g(h(x)))
    """
    def compose_two(f, g):
        def c(x):
            return f(g(x))
        return c
    return functools.reduce(compose_two, funcs)

# Example Using argparse from the Python Standard Lib

[Official Python 3 argparse Docs](https://docs.python.org/3/library/argparse.html)

Let's start from the common example from [RealPython](https://realpython.com/comparing-python-command-line-parsing-libraries-argparse-docopt-click/)

Slightly modified from their example.

In [4]:
def hello(args):
    print('Hello, {0}!'.format(args.name))

def goodbye(args):
    print('Goodbye, {0}!'.format(args.name))

def get_parser(version="0.1.0"):
    parser = argparse.ArgumentParser()
    parser.add_argument('--version', action='version', version=version)
    
    subparsers = parser.add_subparsers()

    hello_parser = subparsers.add_parser('hello')
    hello_parser.add_argument('name')  # add the name argument
    hello_parser.set_defaults(func=hello)  # set the default function to hello

    goodbye_parser = subparsers.add_parser('goodbye')
    goodbye_parser.add_argument('name')
    goodbye_parser.set_defaults(func=goodbye)
    return parser

def runner(argv):
    parser = get_parser()
    args = parser.parse_args(argv)
    return(args.func(args))

#if __name__ == '__main__':
#    args = parser.parse_args()
#    args.func(args)  # call the default function

Let's define a util for testing.

In [5]:
ARGS = [["hello", 'my-name'],
        ["goodbye", 'my-new-name'],
        ["hello", '--help'], # ipython won't play nicely with the argparse useage of sys.exit
        ]

def run_example(runner_func, args=ARGS):
    for arg in args:
        print("Running with args {}".format(arg))
        runner_func(arg)

In [6]:
run_example(runner)

Running with args ['hello', 'my-name']
Hello, my-name!
Running with args ['goodbye', 'my-new-name']
Goodbye, my-new-name!
Running with args ['hello', '--help']
usage: ipykernel_launcher.py hello [-h] name

positional arguments:
  name

optional arguments:
  -h, --help  show this help message and exit


SystemExit: 0

/Users/mkocher/anaconda2/envs/core37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
run_example(runner, [('--version')])

Running with args --version


usage: ipykernel_launcher.py [-h] [--version] {hello,goodbye} ...
ipykernel_launcher.py: error: invalid choice: '-' (choose from 'hello', 'goodbye')


SystemExit: 2

## Comments

It looks like a reasonable interface. However, it's a bit verbose, specifically for the subparser example. There's also a bit of magic with the `.set_defaults(func=my_func)` that isn't particularly obvious. 

Depending on your point of view, this is feature utilizing the `dict` nature of the implementation details, or it's a lackluster design yielding an amorphous interface. Regardless, this ends up with the following pattern to call a specific subparser.

```python
p = get_parser()
pargs = p.parse_args(sys.argv[1:])
return pargs.func(pargs)
```

**Let's see if we can take our Functional Programming Techniques (FPT) and apply them to smooth out some of friction in the argparse interface while also adding more features and extensibility.** 


# Thin Wrapping Layer to argparse

Let's try to smooth out some of the duplication and try to extend the API in "natural" ways, such as to adding a setup hook and error handling mechanism.

### Goals of Each Iteration using FPT

- 1. Cleanup some low hanging duplication
- 2. Add an option to a subparser and add sharing option(s) functionality between subparsers
- 3. Extend: Add logging setup functionality. Add running a post execution hook.
- 4. Extend: Add error handling to map/handle errors to exit code

The overall goal in each iteration is to demonstrate leveraging functions and closures as a core pattern yielding a simple and extensible design. Different use cases would be appropriate for each iteration. For example, Iteration #1 and #2 might be useful for one off use cases, whereas Iteration #3 or #4 would be more appropriate for production code. 

## Iteration #1

For the first iteration let's try to remove some of the duplication and do a bit of cleanup.

Specifically:

- decouple the `argparse` IO layer from the lib code in our package. This will avoid leaking the amorphous parsed args into our lib code.
- remove a bit duplication in the `name` option
- define a utility function to help add a subparser


Let's define an function within `get_parser` to reduce a bit of biolerplate when adding subparsers. 

In [8]:
# Ideally, this should from my lib code and shouldn't
# mix the IO layer from CLI interface

def lib_hello(name):
    print('Hello, {0}!'.format(name))
    
def lib_goodbye(name):
    print('Goodbye, {0}!'.format(name))
    

# CLI. Wrapper funcs of (arg) -> f(a, b, c)
def run_hello(args):
    return lib_hello(args.name)

def run_goodbye(args):
    return lib_goodbye(args.name)

def _add_opt_name(p):
    p.add_argument('name', help="User Name")
    return p


def get_parser(version='0.1.0'):    

    parser = argparse.ArgumentParser(description="Help for Tool")
    parser.add_argument('--version', action='version', version=version)
    subparsers = parser.add_subparsers()
    
    def _add_to_sp(name, add_opts, func):
        p = subparsers.add_parser(name)
        add_opts(p)
        p.set_defaults(func=func)
        return p

    _add_to_sp('hello', _add_opt_name, run_hello)
    _add_to_sp('goodbye', _add_opt_name, run_goodbye)
    
    return parser

def runner(argv):
    p = get_parser()
    pargs = p.parse_args(argv)
    return pargs.func(pargs)

#if __name__ == '__main__':
#    sys.exit(runner(sys.argv[1:]))

In [9]:
run_example(runner)

Running with args ['hello', 'my-name']
Hello, my-name!
Running with args ['goodbye', 'my-new-name']
Goodbye, my-new-name!
Running with args ['hello', '--help']
usage: ipykernel_launcher.py hello [-h] name

positional arguments:
  name        User Name

optional arguments:
  -h, --help  show this help message and exit


SystemExit: 0

## Iteration #2

Let's cleanup a bit more and extend the `hello` subparser by adding a new option (`--num-times`).

In [10]:
def lib_hello(name, num_times=1):
    for _ in range(0, num_times):
        print('Hello, {0}!'.format(name))
    
def run_hello(args):
    return lib_hello(args.name, args.num_times)

def add_opt_hello(p):
    _add_opt_name(p)
    p.add_argument('--num-times', help="Number of times to print hello", default=1, type=int)
    return p


def get_parser(version='0.1.0'):    

    parser = argparse.ArgumentParser(description="Help for Tool")
    parser.add_argument('--version', action='version', version=version)
    subparsers = parser.add_subparsers()
    
    def _add_to_sp(name, add_opts, func):
        p = subparsers.add_parser(name)
        add_opts(p)
        p.set_defaults(func=func)
        return p

    _add_to_sp('hello', add_opt_hello, run_hello)
    _add_to_sp('goodbye', _add_opt_name, run_goodbye)
    
    return parser

def runner(argv):
    p = get_parser()
    pargs = p.parse_args(argv)
    return pargs.func(pargs)

#if __name__ == '__main__':
#    sys.exit(runner(sys.argv[1:]))

In [11]:
run_example(runner, [['hello', 'My-Name','--num-times', '2'], ['hello', '--help']])

Running with args ['hello', 'My-Name', '--num-times', '2']
Hello, My-Name!
Hello, My-Name!
Running with args ['hello', '--help']
usage: ipykernel_launcher.py hello [-h] [--num-times NUM_TIMES] name

positional arguments:
  name                  User Name

optional arguments:
  -h, --help            show this help message and exit
  --num-times NUM_TIMES
                        Number of times to print hello


SystemExit: 0

## Iteration #3 

In this iteration we're going to extend the current API to add two new features.

- Add logging options and logging setup setup (This can be thought of a specific case of a pre-start hook).
- Add enabling of running an epilogue after our lib function has run (This is can a thought of as a post execution hook).

Before we add this fuctionality, let's define a new pattern to help pass functions around more clearly.

Instead of using `None` when passing a function as an arg/kw to a function, let's define an identity or 'null' functions that capture the core interface of the functions that will be passed into our driver function (e.g., `runner`). 

More concretely, instead of this:

```python
def setup_logger(level, output_file=None):
    # implementation omitted for avoid chatter
    pass

def runner(args, setup_logger=None):
    if setup_logger is not None:
        setup_logger(args.level, args.output_file)
```

Well use `typing` from Python 3 combined with a null or identity function to clearly define the interface.


```python
from typing import Optional

def setup_logger_null(level:str, output_file:Optional[str]=None) -> None:
    # implementation omitted for avoid chatter
    pass

def runner(args, setup_logger=setup_logger_null) -> int:
    setup_logger(args.level, args.output_file)
```

Using the approach is useful for a few key reasons.

1. Makes the API extendible by passing the function to the driver that will setup the logger or run the epilogue post execution hook.
2. Provides users with a concrete definition of the function interface (e.g., F(int, int) -> None)) for users to plugin into the API in a clear defined mechanism

Let's add a concrete example with both a `setup_logger` and `run_epilogue` passed into the driver (`runner`) function.

From an OO perspective, this can be thought of as similar to the [Strategy Pattern](https://en.wikipedia.org/wiki/Strategy_pattern).

In [12]:
from typing import Optional

logger = logging.getLogger(__name__)
    
# util funcs for our CLI     
def null_setup_logger(level:str, output_file:Optional[str]=None) -> None:
    pass

def setup_logger(level:str, output_file:Optional[str]=None):
    # implementation omitted
    print("mock setting up logger level={} file={}".format(level, output_file))

def null_run_epilogue(exit_code:int, started_at) -> None:
    pass

def run_epilogue(exit_code:int, started_at):
    dt = datetime.datetime.now() - started_at
    print("completed exit-code={} runtime:{:.2f} sec".format(exit_code, dt.total_seconds()))
    
    
def _add_opt_logging(p):
    f = p.add_argument
    
    f('--level', help="Logging Level", default="INFO")
    f('--log-file', help="Write log to specific output file", default=None)
    return p


def get_parser(version='0.1.0'):    

    parser = argparse.ArgumentParser(description="Help for Tool")
    parser.add_argument('--version', action='version', version=version)
    subparsers = parser.add_subparsers()
    
    def _add_to_sp(name, add_opts, func):
        p = subparsers.add_parser(name)
        # For consistency we want the logging opts to be
        # added to all subparsers
        f = compose(add_opts, _add_opt_logging)
        _ = f(p)
        p.set_defaults(func=func)
        return p

    _add_to_sp('hello', add_opt_hello, run_hello)
    _add_to_sp('goodbye', _add_opt_name, run_goodbye)
    
    return parser

def runner(argv, 
           setup_logger=setup_logger, 
           run_epilogue=run_epilogue) -> int:
    
    exit_code = 1
    started_at = datetime.datetime.now()
    p = get_parser()
    pargs = p.parse_args(argv)
    
    # This is strongly coupled to _add_opt_logging
    # This can be decoupled, by changing the func
    # signature to f(args) -> Unit
    setup_logger(pargs.level, pargs.log_file)
    
    # the pre exe could be passed in
    logger.debug(pargs)
    
    try:
        out = pargs.func(pargs)
        exit_code = 0
    except Exception as ex:
        # we'll fix this in the next iteration
        logger.error(ex)

    run_epilogue(exit_code, started_at)
    return(out)

#if __name__ == '__main__':
#    sys.exit(runner(sys.argv[1:]))

In [13]:
run_example(runner)

Running with args ['hello', 'my-name']
mock setting up logger level=INFO file=None
Hello, my-name!
completed exit-code=0 runtime:0.00 sec
Running with args ['goodbye', 'my-new-name']
mock setting up logger level=INFO file=None
Goodbye, my-new-name!
completed exit-code=0 runtime:0.00 sec
Running with args ['hello', '--help']
usage: ipykernel_launcher.py hello [-h] [--level LEVEL] [--log-file LOG_FILE]
                                   [--num-times NUM_TIMES]
                                   name

positional arguments:
  name                  User Name

optional arguments:
  -h, --help            show this help message and exit
  --level LEVEL         Logging Level
  --log-file LOG_FILE   Write log to specific output file
  --num-times NUM_TIMES
                        Number of times to print hello


SystemExit: 0

Note, because the the `runner` return type has changed (the exception is being caught) and now requires changing the main hook to explicitly exit from `sys`.

```python
if __name__ == '__main__':
    sys.exit(runner(sys.argv[1:]))
```

## Iteration #4

Let's add custom error handling to map Python exceptions to integer return codes using a similar pattern that was used in Iteration #3.

In [14]:
def error_handler(ex, default_error_code=1) -> int:
    """Responsible for mapping the Exception to an Int
    
    Note, this should also write the stacktrace to log or stderr.
    """
    d = {'ZeroDivisionError': 7, 
         'IOError':3,
         'ValueError': 4
        }
    return d.get(ex.__class__.__name__, default_error_code)


def runner(argv, 
           setup_logger=setup_logger,
           error_handler=error_handler,
           run_epilogue=run_epilogue) -> int:
    
    exit_code = 1
    started_at = datetime.datetime.now()
    
    p = get_parser()
    pargs = p.parse_args(argv)
    
    setup_logger(pargs.level, pargs.log_file)
    logger.debug(pargs)
    
    try:
        out = pargs.func(pargs)
        exit_code = 0
    except Exception as ex:
        exit_code = error_handler(ex)

    run_epilogue(exit_code, started_at)
    return(exit_code)

In [15]:
run_example(runner)

Running with args ['hello', 'my-name']
mock setting up logger level=INFO file=None
Hello, my-name!
completed exit-code=0 runtime:0.00 sec
Running with args ['goodbye', 'my-new-name']
mock setting up logger level=INFO file=None
Goodbye, my-new-name!
completed exit-code=0 runtime:0.00 sec
Running with args ['hello', '--help']
usage: ipykernel_launcher.py hello [-h] [--level LEVEL] [--log-file LOG_FILE]
                                   [--num-times NUM_TIMES]
                                   name

positional arguments:
  name                  User Name

optional arguments:
  -h, --help            show this help message and exit
  --level LEVEL         Logging Level
  --log-file LOG_FILE   Write log to specific output file
  --num-times NUM_TIMES
                        Number of times to print hello


SystemExit: 0

# Summary and Misc Comments

We've accomplished a few items

- Broke up the original argparse layer into a core orthagonal components, such as `get_parser`, `runner`. 
- Shared common options using simple functions
- Smoothed out lack of expressiveness (resulting in boilerplate and duplication) of subparsers in argparse by using a few functions as core composable units
- Adding `setup_logger` and `run_epilogue` by passing functions into the driver. 

This concludes Part 3. Hopefully, the examples here provided insight into how to leverage FPT in smooth out duplication and boilerplate when interfacing to existing APIs. 

Best to you and your Python'ing.

## Futher Reading

- Other CLI libs: [click](https://click.palletsprojects.com/en/7.x/) and [docopt](https://github.com/docopt/docopt)
- [Discussion of Sharing Options](https://github.com/pallets/click/issues/108#issuecomment-194465429) using closures and functions in `click`. This is similar to spirit of the thin layer we've added to argparse.  
- [Click has several nice examples](https://github.com/pallets/click/blob/master/click/decorators.py#L92) of decorators and functions are core design principles
- The patterns discussed in Part 3 are used extensively in [pbcommand](https://github.com/mpkocher/pbcommand/blob/master/pbcommand/cli/quick.py#L319)


In [16]:
print("Today is {}".format(datetime.datetime.now()))

Today is 2019-02-21 16:59:28.615358
